<a href="https://colab.research.google.com/github/vzMars/CMP765/blob/main/Homework01_Intelligent_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercise: Intelligent Agent

An **agent** is anything that can be viewed as perceiving its environment through sensors and
acting upon that environment. Let's create an intelligent agent that can navigate through a grid world to reach a target while avoiding obstacles.

### Task 1: Environment Setup

Write a function `create_environment(num_rows, num_cols, num_obstacles)` to create a 2D list as the environment for the agent. The grid should consist of cells, some of which are obstacles (marked as "X") and one of which is the target (marked as "T"). You can represent the grid as a 2D list.
- `num_rows` represents the number of rows.
- `num_cols` represents the number of columns.
- `num_obstacles` represents the number of cells that are obstacles. The function should randomly select the row index and column index for the obstacle cells using [`numpy.random.randint`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randint.html)
- The target cell always appears at the lower-right corner.
- The function should return the 2D list in the end.

In [74]:
import numpy

def create_environment(num_rows, num_cols, num_obstacles):
    # Step 1: Create a 2D list filled with '-'
    grid = []

    for row_index in range(num_rows):
        row = []
        for col_index in range(num_cols):
            row.append('-')
        grid.append(row)

    # Step 2: Add the target
    grid[-1][-1] = 'T'

    # Step 3: Add random obstacles
    obstacles_placed = 0
    while obstacles_placed < num_obstacles:
        row_index = numpy.random.randint(num_rows)
        col_index = numpy.random.randint(num_cols)

        if row_index == 0 and col_index == 0:
            continue

        if grid[row_index][col_index] == '-':
            grid[row_index][col_index] = 'X'
            obstacles_placed += 1

    # Step 4: Return the grid
    return grid

def show_grid(grid):
    for row in grid:
        for col in row:
          print(col, end=" ")
        print()

### Task 2: Agent Class

Create a class `Agent` that represents the intelligent agent.

**Attributes:**
- `self.grid`: the environment
- `self.position`: the current position of the agent, the starting position should always be the upper-left corner.
- `self.target`: the position of the target. The target's position should always be the lower-right corner.

**Methods:**
- `perceive_environment()`: This method allows the agent to perceive its adjacent cells.
- `decide_action()`: Implement a simple decision-making process for your agent to determine the next action (up, down, left, or right).
- `take_action()`: Execute the chosen action and update the agent's position on the grid.

In [75]:
class Agent:

    def __init__(self, grid):
        self.grid = grid
        self.position = (0, 0)
        self.grid[0][0] = 'A'
        self.target = (len(grid) - 1, len(grid[0]) - 1)
        self.path = []
        self.visited = []

    def perceive_environment(self):
        if self.position[0] != 0:
            up = self.grid[self.position[0] - 1][self.position[1]]
        else:
            up = 'W'

        if self.position[0] + 1 < len(self.grid):
            down = self.grid[self.position[0] + 1][self.position[1]]
        else:
            down = 'W'

        if self.position[1] != 0:
            left = self.grid[self.position[0]][self.position[1] - 1]
        else:
            left = 'W'

        if self.position[1] + 1 < len(self.grid[0]):
            right = self.grid[self.position[0]][self.position[1] + 1]
        else:
            right = 'W'

        return [up, down, left, right]

    def decide_action(self):
        up, down, left, right = self.perceive_environment()

        if right == 'T' or down == 'T' or left == 'T' or right == 'T':
            return "End"
        elif right == '-' and (self.position[0], self.position[1] + 1) not in self.visited:
            return 'Right'
        elif down == '-' and (self.position[0] + 1, self.position[1]) not in self.visited:
            return 'Down'
        elif left == '-' and (self.position[0], self.position[1] - 1) not in self.visited:
            return 'Left'
        elif up == '-' and (self.position[0] - 1, self.position[1]) not in self.visited:
            return 'Up'
        else:
            return 'GoBack'

    def take_action(self):
        action = self.decide_action()

        if action == 'Right' or 'Down' or 'Left' or 'Up' or 'End':
            self.grid[self.position[0]][self.position[1]] = '-'

        if action == 'Right':
            self.position = (self.position[0], self.position[1] + 1)
        elif action == 'Down':
            self.position = (self.position[0] + 1, self.position[1])
        elif action == 'Left':
            self.position = (self.position[0], self.position[1] - 1)
        elif action == 'Up':
            self.position = (self.position[0] - 1, self.position[1])
        elif action == 'End':
            self.position = self.target
        elif action == 'GoBack':
            self.path.pop()
            if len(self.path) == 0:
                self.position = (0, 0)
                self.path.append(self.position)
            else:
                self.position = self.path[-1]

        if self.position not in self.visited:
            self.visited.append(self.position)

        if action == 'Right' or 'Down' or 'Left' or 'Up' or 'End':
            self.path.append(self.position)

        self.grid[self.position[0]][self.position[1]] = 'A'

### Task 3: Agent Evaluation

Create a 5x5 grid with 7 obstacles and an instance of the `Agent` class. Simulate the agent's navigation and display the grid at each step to visualize the agent's progress. Use a `for` loop to keep the agent moving until it reaches the target or has taken 100 moves.

In [77]:
grid = create_environment(5, 5, 7)
agent = Agent(grid)
agent.path.append(agent.position)
agent.visited.append(agent.position)

print('Start:')
show_grid(grid)
for x in range(100):
    agent.take_action()
    print()
    print('Move: ', x+1)
    show_grid(grid)
    if agent.position == agent.target:
        print()
        print('Target reached')
        break
if agent.position != agent.target:
    print()
    print('Target not reached')

Start:
A X - X - 
- X - - - 
- - - - X 
X X X - - 
- - - - T 

Move:  1
- X - X - 
A X - - - 
- - - - X 
X X X - - 
- - - - T 

Move:  2
- X - X - 
- X - - - 
A - - - X 
X X X - - 
- - - - T 

Move:  3
- X - X - 
- X - - - 
- A - - X 
X X X - - 
- - - - T 

Move:  4
- X - X - 
- X - - - 
- - A - X 
X X X - - 
- - - - T 

Move:  5
- X - X - 
- X - - - 
- - - A X 
X X X - - 
- - - - T 

Move:  6
- X - X - 
- X - - - 
- - - - X 
X X X A - 
- - - - T 

Move:  7
- X - X - 
- X - - - 
- - - - X 
X X X - A 
- - - - T 

Move:  8
- X - X - 
- X - - - 
- - - - X 
X X X - - 
- - - - A 

Target reached
